In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install wget

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 4.9MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.9MB/s eta 0:00:011
     |████████████████████████████████| 829kB 9.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 122kB 10.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Libraries imported.


In [11]:

venues = defaultdict(list)
categories = {}
coordinates = []

with open('NYC.txt') as nyc_data:
    for venue in nyc_data.readlines():
        data = venue.split('\t')
        
        # get the coordinates for the shop
        coordinates.insert(len(coordinates), [data[4], data[5]])
        
        # store the shop id and the user ids
        if data[1] not in venues:
            venues[data[1]].append(data[0])
        elif data[0] not in venues[data[1]]:
            venues[data[1]].append(data[0])
            
        # store the type of the shop and its id
        if data[1] not in categories:
            categories[data[1]] = data[3]
                
        
# example for 5 places and their visitors according to check-ins
for i in range(5):
    print(list(venues)[i], ": ", len(venues.get(list(venues)[i])), "  --> ", venues.get(list(venues)[i]))
    print()

FileNotFoundError: [Errno 2] No such file or directory: 'NYC.txt'

In [12]:
from collections import defaultdict

venues = defaultdict(list)
categories = {}
coordinates = []

with open('data/NYC.txt') as nyc_data:
    for venue in nyc_data.readlines():
        data = venue.split('\t')
        
        # get the coordinates for the shop
        coordinates.insert(len(coordinates), [data[4], data[5]])
        
        # store the shop id and the user ids
        if data[1] not in venues:
            venues[data[1]].append(data[0])
        elif data[0] not in venues[data[1]]:
            venues[data[1]].append(data[0])
            
        # store the type of the shop and its id
        if data[1] not in categories:
            categories[data[1]] = data[3]
                
        
# example for 5 places and their visitors according to check-ins
for i in range(5):
    print(list(venues)[i], ": ", len(venues.get(list(venues)[i])), "  --> ", venues.get(list(venues)[i]))
    print()

FileNotFoundError: [Errno 2] No such file or directory: 'data/NYC.txt'

In [13]:
# define the dataframe columns
column_names = ['VenueID', 'CategoryName', 'Visitor Count', 'Latitude', 'Longitude'] 

# instantiate the dataframe
venue_data = pd.DataFrame(columns=column_names)

venue_data

,VenueID,CategoryName,Visitor Count,Latitude,Longitude


In [14]:
# loop through the data and fill the dataframe one row at a time.
for idx in range(0, 2000):
    venue_id = list(venues)[idx]
    coords = list(coordinates)[idx]
    visitorCount = len(venues.get(venue_id))
    venue_type = categories.get(venue_id)


    venue_data = venue_data.append({ 'CategoryName': venue_type,
                                     'VenueID': venue_id,
                                     'Visitor Count': visitorCount,
                                     'Latitude': coords[0],
                                     'Longitude': coords[1]}, ignore_index=True)

IndexError: list index out of range

In [15]:

# Quickly examine the resulting dataframe.
venue_data.head()

,VenueID,CategoryName,Visitor Count,Latitude,Longitude


In [16]:
print('The dataframe has {} venues with total of {} visitors.'.format(
        len(venue_data['VenueID'].unique()),
        sum(venue_data['Visitor Count'])
    )
)

The dataframe has 0 venues with total of 0 visitors.


In [17]:
import math as Math
def pointInCircle(lat0, lon0, r, lat, lon):  
    C = 40075.04                                               # Earth circumference
    A = 360*r/C                                                # semi-minor in north-south direction 
    B = A/Math.cos(Math.radians(float(lat0)));                 # semi-major in east-west direction
    return Math.pow((float(lat)-float(lat0))/A, 2) + Math.pow((float(lon)-float(lon0))/B, 2) < 1;

In [ ]:
someCategories = ['Food & Drink Shop', 'Electronics Store', 'Coffee Shop', 'Restaurant', 'Arts & Crafts Store', 'Gastropub', 
                  'Mobile Phone Shop', 'Café', 'Automative Shop', 'American Restaurant', 'Food & Drink Shop', 'Burger Joint',
                  'Mexican Restaurant', 'Sandwich Place', 'Clothing Store', 'Ice Cream Shop', 'Pizza Place', 'Jewelry Store', 
                  'Soup Place', 'Tattoo Parlor', 'Deli / Bodega', 'Diner', 'Salon / Barbershop', 'Laundry Service', 'Bar', 
                  'Gym / Fitness Center', 'Hotel', 'Music Venue', 'BBQ Joint', 'Bookstore', 'Drugstore / Pharmacy',
                  'Sporting Goods Shop', 'Bakery', 'Fast Food Restaurant', 'Chinese Restaurant', 'Theater', 'Movie Theater', 
                  'Sushi Restaurant', 'Miscellaneous Shop', 'French Restaurant', 'Seafood Restaurant', 'Fried Chicken Joint', 
                  'Italian Restaurant', 'Toy / Game Store', 'Vegetarian / Vegan Restaurant', 'Donut Shop', 'German Restaurant', 
                  'Bowling Alley', 'Beer Garden', 'Candy Store', 'Bagel Shop', 'Cuban Restaurant', 'Cupcake Shop', 
                  'Breakfast Spot', 'Hardware Store', 'Japanese Restaurant',  'Latin American Restaurant', 'Spanish Restaurant',
                  'Spa / Massage', 'Middle Eastern Restaurant', 'Malaysian Restaurant', 'Record Shop', 'Wings Joint', 
                  'Gas Station / Garage', 'Asian Restaurant', 'Burrito Place', 'Thai Restaurant', 'Salad Place', 
                  'Ramen /  Noodle House', 'Automotive Shop', 'Convenience Store', 'Tea Room',  'Indian Restaurant', 
                  'Thrift / Vintage Store', 'Paper / Office Supplies Store', 'Cosmetics Shop', 'Southern / Soul Food Restaurant',
                  'Smoke Shop', 'Snack Place', 'Furniture / Home Store', 'Caribbean Restaurant', 'Video Game Store', 'Steakhouse',
                  'Greek Restaurant', 'Dumpling Restaurant', 'Mediterranean Restaurant', 'African Restaurant', 'Taco Place', 
                  'Jewelry Store', 'Hot Dog Joint', 'South American Restaurant', 'Winery']
visitForCategories = {}
maxVisited = ""

# count the number of visitors for categories
for idx in range(0, 2000):
    venue_id = list(venues)[idx]
    visitorCount = len(venues.get(venue_id))
    venue_type = categories.get(venue_id)
    
    if venue_type not in visitForCategories:
        visitForCategories[venue_type] = visitorCount
    else: 
        visitForCategories[venue_type] = visitForCategories[venue_type]+visitorCount
        
# visitForCategories
sorted_dict = sorted(visitForCategories.items(), key=lambda x: x[1], reverse=True)
for v in sorted_dict:
    if v[0] in someCategories:
        maxVisited = v[0]
        break
        
# category names with their visit number
sorted_dict = sorted(visitForCategories.items(), key=lambda x: x[1], reverse=True)
print(sorted_dict)

print()

# Max visited category
print("'" + maxVisited + "'", "is the most visited commercial category according to given data.")